In [1]:
import json
import os
import re
from collections import Counter
from collections import defaultdict
from pprint import pprint as pp

In [2]:
# Берем заранее составленный список героев (собранный автоматически по рабочему корпусу и вычищенный вручную) 
# и заменяем пробелы в составных именах на _ чтобы избежать появления связей вида имя-патроним, имя-прозвище и т.д.
# (т.е. мы не хотим получить связь Conchobar-Ness из составного имени Conchobar mac Nessa)

with open('characterlist.txt', 'r', encoding='utf-8') as cl:
    characters = set([line.lower().strip() for line in cl])

charDict = {}
for char in characters:
    charDict[char] = re.sub(r'\s', r'_', char)

In [4]:
# Проделываем ту же замену во всех текстах рабочего корпуса (уже лемматизированных)
# Затем проводим простейшую токенизацию (разбиваем по пробелам) и смотрим объем корпуса в токенах

path = 'C:\\Users\\Oksana\\Dropbox\\Библиотека\\Вышка\\OldIrish\\texts\\ulster\\processed\\lemmatized'
files = [name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))]
terms = []
for file in files:
    with open(os.path.join(path, file), 'r', encoding='utf-8') as f:
        text = f.read()
        for char in characters:
            if char in text:
                text = re.sub(char, charDict[char], text)
        terms += text.split()

print(len(terms)) 

151750


In [5]:
# Класс, который будет считать совместную встречаемость имен в окне размером 5

class Matcher():
    def __init__(self, phrases):
        phrase_pattern = "|".join("(?:{})".format(phrase) for phrase in phrases)
        gap_pattern = r"\W+(?:\w+\W+){0,5}?"
        full_pattern = "({0}){1}({0})".format(phrase_pattern, gap_pattern)

        self.regex = re.compile(full_pattern)

    def match(self, doc):
        return self.regex.findall(doc)

matcher = Matcher(charDict.values())
com = defaultdict(lambda: defaultdict(int))

In [6]:
# Фильтруем результаты и делаем матрицу совместной встречаемости

matches = matcher.match(' '.join(terms))
filteredMatches = []
for el in matches:
    el = [name for name in el if len(name) != 0]
    el = [name for name in el if el[0] != el[1]]
    filteredMatches.append(tuple(set(el)))

counts = Counter(filteredMatches)

for k, v in counts.items():
    if len(k) != 0:
        com[k[0]][k[1]] += v

In [7]:
# Записываем ее в файл на будущее

with open('pair_counts_5.json', 'w', encoding='utf-8') as p:
    json.dump(com, p, sort_keys=True, ensure_ascii=False)

In [9]:
# Привели имена в файле обратно в красивый вид в Notepad++ (так быстрее), вернули капитализацию, так что заново его прочитаем

with open('pair_counts_5.json', 'r', encoding='utf-8') as f:
    characters = json.load(f)

In [14]:
pp(characters['Deirdriu'])

{'Cellach': 1,
 'Cet': 1,
 'Erc': 1,
 'Leborcham': 7,
 'Lóeg': 1,
 'Macha': 1,
 'Nem': 2,
 'Uisliu': 1}


In [22]:
# Осталось превратить все это в граф

import networkx as nx
G=nx.Graph()

for k, v in characters.items():
    for link, weight in v.items():
        G.add_weighted_edges_from([(k, link, weight)])

In [29]:
# Проверяем, что получилось (смотрим самые сильные связи)

for n,nbrs in G.adjacency_iter():
    for nbr,eattr in nbrs.items():
        data=eattr['weight']
        if data > 10:
            print('(%s, %s, %d)' % (n,nbr,data))

(Ailill, Cú Roí, 14)
(Ailill, Medb, 133)
(Ailill, Sencha, 19)
(Ailill, Fergus, 54)
(Ailill, Cú Culand, 13)
(Lóegaire, Conall, 12)
(Cú Culand, Conchobar, 14)
(Cú Culand, Ailill, 13)
(Cú Culand, Medb, 25)
(Cú Culand, Láeg, 24)
(Cú Culand, Fergus, 33)
(Cormac, Conchobar, 17)
(Sencha, Ailill, 19)
(Medb, Conchobar, 12)
(Medb, Ailill, 133)
(Medb, Cú Culand, 25)
(Medb, Fergus, 36)
(Fíngen, Cethern, 14)
(Conchobar, Cathbad, 14)
(Conchobar, Fergus, 20)
(Conchobar, Medb, 12)
(Conchobar, Cú Culand, 14)
(Conchobar, Cormac, 17)
(Cethern, Fíngen, 14)
(Fergus, Ailill, 54)
(Fergus, Medb, 36)
(Fergus, Conchobar, 20)
(Fergus, Cú Culand, 33)
(Láeg, Cú Culand, 24)
(Cú Roí, Ailill, 14)
(Cathbad, Conchobar, 14)
(Conall, Lóegaire, 12)


In [36]:
# Осталось только экспортировать в формат, поддерживаемый Gephi, чтобы потом экспортировать в Sigma

nx.write_graphml(G, 'Ulster.graphml', encoding='utf-8')

In [31]:
# Попробуем нарисовать

import matplotlib.pyplot as plt
import seaborn

nx.draw_random(G)
plt.show()

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [35]:
G.neighbors('Deirdriu')

['Cellach', 'Leborcham', 'Lóeg', 'Cet', 'Nem', 'Macha', 'Erc', 'Uisliu']